## MTH6412B - Projet Phase 1
#### Victor Darleguy - Nathan Allaire

### Question 1) 

In [ ]:
include("node.jl")
include("graph.jl")
include("read_stsp.jl")

plot_graph

### Question 2)

Création du type `Edge` :

In [ ]:
"""
Type abstrait dont d'autres types d'arêtes dériveront.
"""
abstract type AbstractEdge{T, U} end

"""
Type représentant les arêtes d'un graphe.

Exemple:

    node1 = Node("Joe", 3.14)
    node2 = Node("Steve", exp(1))
    edge = Edge(node1, node2)       # Arête non pondérée
    edge = Edge(node1, node2, 5.6)  # Arête pondérée avec un poids de 5.6
"""
mutable struct Edge{T, U} <: AbstractEdge{T, U}  
  start_node::Node{T}                            # Le nœud de départ de l'arête.
  end_node::Node{T}                              # Le nœud d'arrivée de l'arête.
  weight::U                                      # Le poids de l'arête. Ce champ peut être générique 
end                                              #  et peut être nothing s'il s'agit d'une arête non pondérée.

# Constructeur pour les arêtes non pondérées
Edge(s::Node{T}, e::Node{T}) where T = Edge(s, e, nothing)

# on présume que toutes les arêtes dérivant d'AbstractEdge
# posséderont des champs `start_node`, `end_node` et `weight`.

"""Renvoie le noeud de départ de l'arête."""
start_node(edge::AbstractEdge) = edge.start_node

"""Renvoie le noeud d'arrivée de l'arête."""
end_node(edge::AbstractEdge) = edge.end_node

"""Renvoie le poids de l'arête."""
weight(edge::AbstractEdge) = edge.weight

"""Affiche une arête."""
function show(edge::AbstractEdge)
    if edge.weight === nothing
        println("Edge from ", name(start_node(edge)), " to ", name(end_node(edge)))
    else
        println("Edge from ", name(start_node(edge)), " to ", name(end_node(edge)), ", weight: ", weight(edge))
    end
end  

Base.show

Exemple d'utilisation du type `Edge`:

In [ ]:
edge_test = Edge(Node("a", 1.0), Node("b", 2.4))

Edge{Float64, Nothing}(Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), nothing)

In [ ]:
show(edge_test)

Edge from a to b


In [ ]:
using Test

@testset "Edge Tests" begin
    node1 = Node("A", 1.0)
    node2 = Node("B", 2.0)
    edge_unweighted = Edge(node1, node2)
    edge_weighted = Edge(node1, node2, 3.0)

    @test start_node(edge_unweighted) == node1
    @test end_node(edge_unweighted) == node2
    @test weight(edge_unweighted) == nothing

    @test start_node(edge_weighted) == node1
    @test end_node(edge_weighted) == node2
    @test weight(edge_weighted) == 3.0
    
end

Test Summary: | Pass  Total  Time
Edge Tests    |    6      6  0.2s


Test.DefaultTestSet("Edge Tests", Any[], 6, false, false, true, 1.696896904163e9, 1.696896904362e9, false)

### Question 3)

Extension du type `Graph` pour contenir les arêtes du graphe

In [ ]:
# Définir un nouveau type `ExtendedGraph` 
mutable struct ExtendedGraph{T} <: AbstractGraph{T}
  name::String
  nodes::Vector{Node{T}}
  edges::Vector{Edge{T}}
end

# Constructeur pour ExtendedGraph
ExtendedGraph(name::String, nodes::Vector{Node{T}}) where T = ExtendedGraph(name, nodes, Edge{T}[])

# Fonction pour ajouter un noeud à un AbstractGraph : 

"""Ajoute un noeud à un ExtendedGraph"""
# Arguments : 
# - `graph::AbstractGraph{T}` : Le graphe auquel ajouter le noeud.
# - `node::Node{T}` : Le noeud à ajouter au graphe.

function add_node!(graph::AbstractGraph{T}, node::Node{T}) where T
  push!(graph.nodes, node)
  graph
end

# Fonction pour ajouter une arête à ExtendedGraph
"""Ajoute une arête à un ExtendedGraph"""
# graph::ExtendedGraph{T} : Le graphe auquel ajouter l'arête.
# start_node::Node{T} : Le nœud de départ de l'arête.
# end_node::Node{T} : Le nœud d'arrivée de l'arête.
# weight::T (optionnel) : Le poids de l'arête (par défaut nothing si non spécifié).

function add_edge!(graph::ExtendedGraph{T}, start_node::Node{T}, end_node::Node{T}, weight = nothing) where {T}
  # Vérifier si les nœuds existent déjà dans le graphe
  if start_node ∉ graph.nodes || end_node ∉ graph.nodes
      throw(ArgumentError("One or both nodes are not part of the graph"))
  end

  # Créer la nouvelle arête
  new_edge = Edge(start_node, end_node, weight)
  
  # Vérifier si l'arête existe déjà
  for edge in graph.edges
      if (edge.start_node == start_node && edge.end_node == end_node) || 
         (edge.start_node == end_node && edge.end_node == start_node)
         return  # simplement retourner si l'arête existe déjà, évitant une erreur

      end
  end

  # Ajouter la nouvelle arête au graphe
  push!(graph.edges, new_edge)
end


add_edge! (generic function with 2 methods)

In [ ]:
ext_graph = ExtendedGraph("graph_test", [Node("a", 1.0), Node("b", 2.4), Node("c", 3.0)])

ExtendedGraph{Float64}("graph_test", Node{Float64}[Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), Node{Float64}("c", 3.0)], Edge{Float64}[])

In [ ]:
add_edge!(ext_graph, ext_graph.nodes[1], ext_graph.nodes[2])

1-element Vector{Edge{Float64}}:
 Edge{Float64, Nothing}(Node{Float64}("a", 1.0), Node{Float64}("b", 2.4), nothing)

In [ ]:
using Test

# Créer un graphe vide pour les tests
graph = ExtendedGraph("Graphe de test", Node{Float64}[])

# Test pour add_node!
@testset "Tests pour add_node!" begin
    # Vérifier que le graphe est initialement vide
    @test length(graph.nodes) == 0
    
    # Ajouter un nœud au graphe
    node = Node("Noeud 1", 3.14)
    add_node!(graph, node)
    
    # Vérifier que le nœud a été ajouté
    @test length(graph.nodes) == 1
    @test graph.nodes[1] === node
end

# Test pour add_edge!
@testset "Tests pour add_edge!" begin
    # Ajouter quelques nœuds au graphe
    node1 = Node("Noeud 1", 3.14)
    node2 = Node("Noeud 2", 2.71)
    add_node!(graph, node1)
    add_node!(graph, node2)
    
    # Ajouter une arête au graphe
    add_edge!(graph, node1, node2, 1.0)
    
    # Vérifier que l'arête a été ajoutée
    @test length(graph.edges) == 1
    edge = graph.edges[1]
    @test start_node(edge) === node1
    @test end_node(edge) === node2
    @test weight(edge) == 1.0
end

Test Summary:        | Pass  Total  Time
Tests pour add_node! |    3      3  0.0s
Test Summary:        | Pass  Total  Time
Tests pour add_edge! |    4      4  

Test.DefaultTestSet("Tests pour add_edge!", Any[], 4, false, false, true, 1.696896905841e9, 1.696896905862e9, false)

0.0s


### Question 4)

Implémentation de `show` pour un objet de type `ExtendedGraph`

In [ ]:
# Affichage de ExtendedGraph
"""Extension de la fonction `show`pour un ExtendedGraph"""
function show(graph::ExtendedGraph{T}) where {T}
  println("Graph ", name(graph), " has ", nb_nodes(graph), " nodes and ", length(graph.edges), " edges.")
  println("Nodes:")
  for node in nodes(graph)
      show(node)
  end
  println("Edges:")
  for edge in graph.edges
      show(edge)
  end
end

Base.show

In [ ]:
show(ext_graph)

Graph graph_test has 3 nodes and 1 edges.
Nodes:
Node a, data: 1.0
Node b, data: 2.4
Node c, data: 3.0
Edges:
Edge from a to b


### Question 5)

La méthode `read_edges()` affiche les arêtes des graphes, mais ignore leurs poids.

In [ ]:
header = read_header("instances/stsp/bayg29.tsp")
read_edges(header, "instances/stsp/bayg29.tsp")

406-element Vector{Any}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 6)
 (1, 7)
 (1, 8)
 (1, 9)
 (1, 10)
 (1, 11)
 ⋮
 (25, 27)
 (25, 28)
 (25, 29)
 (26, 27)
 (26, 28)
 (26, 29)
 (27, 28)
 (27, 29)
 (28, 29)

Modification de la méthode `read_edges()` :

In [ ]:
"""Analyse un fichier .tsp et renvoie l'ensemble des arêtes sous la forme d'un tableau."""
#  Arguments
# - `header::Dict{String, String}` : Le dictionnaire contenant les informations de l'en-tête du fichier .tsp.
# - `filename::String` : Le nom du fichier .tsp à analyser.

#  Résultats
# - `edges::Vector{Tuple{Int, Int, Float64}}` : Un tableau d'arêtes représentées sous forme de tuples (nœud de départ, nœud d'arrivée, poids).

function read_edges(header::Dict{String, String}, filename::String)

  edges = []
  edge_weight_format = header["EDGE_WEIGHT_FORMAT"]
  known_edge_weight_formats = ["FULL_MATRIX", "UPPER_ROW", "LOWER_ROW",
  "UPPER_DIAG_ROW", "LOWER_DIAG_ROW", "UPPER_COL", "LOWER_COL",
  "UPPER_DIAG_COL", "LOWER_DIAG_COL"]

  if !(edge_weight_format in known_edge_weight_formats)
      @warn "unknown edge weight format" edge_weight_format
      return edges, edge_weights
  end

  file = open(filename, "r")
  dim = parse(Int, header["DIMENSION"])
  edge_weight_section = false
  k = 0
  n_edges = 0
  i = 0
  n_to_read = n_nodes_to_read(edge_weight_format, k, dim)
  flag = false

  for line in eachline(file)
      line = strip(line)
      if !flag
          if occursin(r"^EDGE_WEIGHT_SECTION", line)
              edge_weight_section = true
              continue
          end

          if edge_weight_section
              data = split(line)
              n_data = length(data)
              start = 0
              while n_data > 0
                  n_on_this_line = min(n_to_read, n_data)

                  for j = start : start + n_on_this_line - 1
                    # Les lignes suivantes ont été modifiées pour tenir compte du poids des arêtes
                    n_edges = n_edges + 1
                    edge_weight_value = parse(Float64, data[start+j+1])
                    if edge_weight_format in ["UPPER_ROW", "LOWER_COL"]
                      edge = (k+1, i+k+2, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_DIAG_ROW", "LOWER_DIAG_COL"]
                      edge = (k+1, i+k+1, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_COL", "LOWER_ROW"]
                      edge = (i+k+2, k+1, edge_weight_value)
                    elseif edge_weight_format in ["UPPER_DIAG_COL", "LOWER_DIAG_ROW"]
                      edge = (i+1, k+1, edge_weight_value)
                    elseif edge_weight_format == "FULL_MATRIX"
                      edge = (k+1, i+1, edge_weight_value)
                    else
                      warn("Unknown format - function read_edges")
                    end
                    push!(edges, edge)
                    i += 1
                  end

                  n_to_read -= n_on_this_line
                  n_data -= n_on_this_line

                  if n_to_read <= 0
                      start += n_on_this_line
                      k += 1
                      i = 0
                      n_to_read = n_nodes_to_read(edge_weight_format, k, dim)
                  end

                  if k >= dim
                      n_data = 0
                      flag = true
                  end
              end
          end
      end
  end
  close(file)
  return edges
end


read_edges (generic function with 1 method)

In [ ]:
r = read_edges(header, "instances/stsp/bayg29.tsp")

406-element Vector{Any}:
 (1, 2, 97.0)
 (1, 3, 205.0)
 (1, 4, 139.0)
 (1, 5, 86.0)
 (1, 6, 60.0)
 (1, 7, 220.0)
 (1, 8, 65.0)
 (1, 9, 111.0)
 (1, 10, 115.0)
 (1, 11, 227.0)
 ⋮
 (25, 27, 120.0)
 (25, 28, 205.0)
 (25, 29, 270.0)
 (26, 27, 213.0)
 (26, 28, 145.0)
 (26, 29, 36.0)
 (27, 28, 94.0)
 (27, 29, 217.0)
 (28, 29, 162.0)

### Question 6)

L'instance `bays29.tsp` est une matrice pleine symétrique dont les poids sont donnés au format `EXPLICIT`

In [ ]:
header = read_header("instances/stsp/bays29.tsp")
nodes_graph = read_nodes(header, "instances/stsp/bays29.tsp")
edges_graph = read_edges(header, "instances/stsp/bays29.tsp")

841-element Vector{Any}:
 (1, 1, 0.0)
 (1, 2, 107.0)
 (1, 3, 241.0)
 (1, 4, 190.0)
 (1, 5, 124.0)
 (1, 6, 80.0)
 (1, 7, 316.0)
 (1, 8, 76.0)
 (1, 9, 152.0)
 (1, 10, 157.0)
 ⋮
 (29, 21, 108.0)
 (29, 22, 332.0)
 (29, 23, 342.0)
 (29, 24, 218.0)
 (29, 25, 350.0)
 (29, 26, 39.0)
 (29, 27, 263.0)
 (29, 28, 199.0)
 (29, 29, 0.0)

Nous avons créé un vecteur contenant les arêtes et les poids des arêtes. Construisons à présent l'objet `Graph` correspondant :  

In [ ]:
"""Construit un graphe à partir d'une instance TSP symétrique dont les poids sont donnés au format EXPLICIT et renvoie l'objet Graph associé"""
#  Arguments
# - `instance::String` : Le chemin vers le fichier TSP contenant les données de l'instance.
# - `title::String` : Le titre du graphe construit.
# Examples
```
julia> build_graph("instances/stsp/bays29.tsp", "Graph_Test")

Graph Graph_Test has 29 nodes and 435 edges.
Nodes:
Node 1, data: [1150.0, 1760.0]
Node 2, data: [630.0, 1660.0]
# ... (d'autres nœuds et arêtes)
Edge from 28 to 29, weight: 199.0
Edge from 29 to 29, weight: 0.0
```

function build_graph(instance::String, title::String)
  
  header = read_header(instance)
  if header["EDGE_WEIGHT_TYPE"] != "EXPLICIT"
    @warn "weight type format must be `EXPLICIT`"
    return "Current format is :", header["EDGE_WEIGHT_TYPE"]
  end
  if header["EDGE_WEIGHT_FORMAT"] != "FULL_MATRIX"
    @warn "edge weight format must be `FULL_MATRIX`"
    return "Current format is :", header["EDGE_WEIGHT_FORMAT"]
  end

  nodes_graph = read_nodes(header, instance)
  edges_graph = read_edges(header, instance)

  if isempty(nodes_graph)
    for i = 1:parse(Int64, header["DIMENSION"])
      dict_intermediaire = Dict(i => [])
      merge!(nodes_graph, dict_intermediaire)
    end
  end


  graph_built = ExtendedGraph(title, [Node("1", nodes_graph[1])])
  for i=2:length(nodes_graph)
    node = Node(string(i), nodes_graph[i])
    add_node!(graph_built, node)
  end

  for i=1:length(edges_graph)
    s_node = graph_built.nodes[edges_graph[i][1]]
    e_node = graph_built.nodes[edges_graph[i][2]]
    w_edge = edges_graph[i][3]
    add_edge!(graph_built, s_node, e_node, w_edge)
  end
  show(graph_built)
end

build_graph

On test le `graph_built()` avec le fichier bays29.tsp car il s'agit d'une instance de TSP symétrique dont les poids sont donnés au format  EXPLICIT. 

Il est important de noter que nous n'avons considéré que les matrices comme étant symétriques de plein rang (FULL_MATRIX). Il aurait été possible de prendre en compte les matrices triangulaires supérieures (UPPER_ROW) et inférieures (LOWER_ROW) si l'énoncé avait spécifié qu'il s'agissait de matrices symétriques.

In [ ]:
build_graph("/instances/stsp/bays29.tsp", "Graph_Test")

Graph Graph_Test has 29 nodes and 435 edges.
Nodes:
Node 1, data: [1150.0, 1760.0]
Node 2, data: [630.0, 1660.0]
Node 3, data: [40.0, 2090.0]
Node 4, data: [750.0, 1100.0]
Node 5, data: [750.0, 2030.0]
Node 6, data: [1030.0, 2070.0]
Node 7, data: [1650.0, 650.0]
Node 8, data: [1490.0, 1630.0]
Node 9, data: [790.0, 2260.0]
Node 10, data: [710.0, 1310.0]
Node 11, data: [840.0, 550.0]
Node 12, data: [1170.0, 2300.0]
Node 13, data: [970.0, 1340.0]
Node 14, data: [510.0, 700.0]
Node 15, data: [750.0, 900.0]
Node 16, data: [1280.0, 1200.0]
Node 17, data: [230.0, 590.0]
Node 18, data: [460.0, 860.0]
Node 19, data: [1040.0, 950.0]
Node 20, data: [590.0, 1390.0]
Node 21, data: [830.0, 1770.0]
Node 22, data: [490.0, 500.0]
Node 23, data: [1840.0, 1240.0]
Node 24, data: [1260.0, 1500.0]
Node 25, data: [1280.0, 790.0]
Node 26, data: [490.0, 2130.0]
Node 27, data: [1460.0, 1420.0]
Node 28, data: [1260.0, 1910.0]
Node 29, data: [360.0, 1980.0]
Edges:
Edge from 1 to 1, weight: 0.0
Edge from 1 to 2, w

13 to 14, weight: 186.0
Edge from 13 to 15, weight: 117.0
Edge from 13 to 16, weight: 75.0
Edge from 13 to 17, weight: 231.0
Edge from 13 to 18, weight: 165.0
Edge from 13 to 19, weight: 81.0
Edge from 13 to 20, weight: 85.0
Edge from 13 to 21, weight: 92.0
Edge from 13 to 22, weight: 230.0
Edge from 13 to 23, weight: 184.0
Edge from 13 to 24, weight: 74.0
Edge from 13 to 25, weight: 150.0
Edge from 13 to 26, weight: 208.0
Edge from 13 to 27, weight: 104.0
Edge from 13 to 28, weight: 158.0
Edge from 13 to 29, weight: 206.0
Edge from 14 to 14, weight: 0.0
Edge from 14 to 15, weight: 69.0
Edge from 14 to 16, weight: 191.0
Edge from 14 to 17, weight: 59.0
Edge from 14 to 18, weight: 35.0
Edge from 14 to 19, weight: 125.0
Edge from 14 to 20, weight: 167.0
Edge from 14 to 21, weight: 255.0
Edge from 14 to 22, weight: 44.0
Edge from 14 to 23, weight: 309.0
Edge from 14 to 24, weight: 245.0
Edge from 14 to 25, weight: 169.0
Edge from 14 to 26, weight: 327.0
Edge from 14 to 27, weight: 246.0
E

Edge from 21 to 25, weight: 236.0
Edge from 21 to 26, weight: 110.0
Edge from 21 to 27, weight: 149.0
Edge from 21 to 28, weight: 97.0
Edge from 21 to 29, weight: 108.0
Edge from 22 to 22, weight: 0.0
Edge from 22 to 23, weight: 353.0
Edge from 22 to 24, weight: 289.0
Edge from 22 to 25, weight: 213.0
Edge from 22 to 26, weight: 371.0
Edge from 22 to 27, weight: 290.0
Edge from 22 to 28, weight: 379.0
Edge from 22 to 29, weight: 332.0
Edge from 23 to 23, weight: 0.0
Edge from 23 to 24, weight: 121.0
Edge from 23 to 25, weight: 162.0
Edge from 23 to 26, weight: 345.0
Edge from 23 to 27, weight: 80.0
Edge from 23 to 28, weight: 189.0
Edge from 23 to 29, weight: 342.0
Edge from 24 to 24, weight: 0.0
Edge from 24 to 25, weight: 154.0
Edge from 24 to 26, weight: 220.0
Edge from 24 to 27, weight: 41.0
Edge from 24 to 28, weight: 93.0
Edge from 24 to 29, weight: 218.0
Edge from 25 to 25, weight: 0.0
Edge from 25 to 26, weight: 352.0
Edge from 25 to 27, weight: 147.0
Edge from 25 to 28, weight

Si le fichier satisfait les conditions de type et format, mais qu'il ne possède pas de valeurs associées au noeud, on construit un graphe composé de noeuds vides

In [ ]:
build_graph("/instances/stsp/swiss42.tsp", "Graph_Test")

Graph Graph_Test has 42 nodes and 903 edges.
Nodes:
Node 1, data: Float64[]
Node 2, data: Float64[]
Node 3, data: Float64[]
Node 4, data: Float64[]
Node 5, data: Float64[]
Node 6, data: Float64[]
Node 7, data: Float64[]
Node 8, data: Float64[]
Node 9, data: Float64[]
Node 10, data: Float64[]
Node 11, data: Float64[]
Node 12, data: Float64[]
Node 13, data: Float64[]
Node 14, data: Float64[]
Node 15, data: Float64[]
Node 16, data: Float64[]
Node 17, data: Float64[]
Node 18, data: Float64[]
Node 19, data: Float64[]
Node 20, data: Float64[]
Node 21, data: Float64[]
Node 22, data: Float64[]
Node 23, data: Float64[]
Node 24, data: Float64[]
Node 25, data: Float64[]
Node 26, data: Float64[]
Node 27, data: Float64[]
Node 28, data: Float64[]
Node 29, data: Float64[]
Node 30, data: Float64[]
Node 31, data: Float64[]
Node 32, data: Float64[]
Node 33, data: Float64[]
Node 34, data: Float64[]
Node 35, data: Float64[]
Node 36, data: Float64[]
Node 37, data: Float64[]
Node 38, data: Float64[]
Node 39

5 to 39, weight: 129.0
Edge from 5 to 40, weight: 127.0
Edge from 5 to 41, weight: 161.0
Edge from 5 to 42, weight: 95.0
Edge from 6 to 6, weight: 0.0
Edge from 6 to 7, weight: 23.0
Edge from 6 to 8, weight: 55.0
Edge from 6 to 9, weight: 96.0
Edge from 6 to 10, weight: 123.0
Edge from 6 to 11, weight: 78.0
Edge from 6 to 12, weight: 75.0
Edge from 6 to 13, weight: 62.0
Edge from 6 to 14, weight: 36.0
Edge from 6 to 15, weight: 56.0
Edge from 6 to 16, weight: 66.0
Edge from 6 to 17, weight: 63.0
Edge from 6 to 18, weight: 95.0
Edge from 6 to 19, weight: 37.0
Edge from 6 to 20, weight: 34.0
Edge from 6 to 21, weight: 137.0
Edge from 6 to 22, weight: 174.0
Edge from 6 to 23, weight: 156.0
Edge from 6 to 24, weight: 129.0
Edge from 6 to 25, weight: 224.0
Edge from 6 to 26, weight: 90.0
Edge from 6 to 27, weight: 15.0
Edge from 6 to 28, weight: 59.0
Edge from 6 to 29, weight: 75.0
Edge from 6 to 30, weight: 96.0
Edge from 6 to 31, weight: 103.0
Edge from 6 to 32, weight: 105.0
Edge from 6 

184.0
Edge from 15 to 25, weight: 273.0
Edge from 15 to 26, weight: 146.0
Edge from 15 to 27, weight: 71.0
Edge from 15 to 28, weight: 95.0
Edge from 15 to 29, weight: 113.0
Edge from 15 to 30, weight: 138.0
Edge from 15 to 31, weight: 138.0
Edge from 15 to 32, weight: 81.0
Edge from 15 to 33, weight: 107.0
Edge from 15 to 34, weight: 159.0
Edge from 15 to 35, weight: 146.0
Edge from 15 to 36, weight: 132.0
Edge from 15 to 37, weight: 113.0
Edge from 15 to 38, weight: 32.0
Edge from 15 to 39, weight: 200.0
Edge from 15 to 40, weight: 209.0
Edge from 15 to 41, weight: 243.0
Edge from 15 to 42, weight: 171.0
Edge from 16 to 16, weight: 0.0
Edge from 16 to 17, weight: 11.0
Edge from 16 to 18, weight: 54.0
Edge from 16 to 19, weight: 103.0
Edge from 16 to 20, weight: 46.0
Edge from 16 to 21, weight: 130.0
Edge from 16 to 22, weight: 230.0
Edge from 16 to 23, weight: 198.0
Edge from 16 to 24, weight: 192.0
Edge from 16 to 25, weight: 279.0
Edge from 16 to 26, weight: 155.0
Edge from 16 to 2

90.0
Edge from 30 to 30, weight: 0.0
Edge from 30 to 31, weight: 21.0
Edge from 30 to 32, weight: 128.0
Edge from 30 to 33, weight: 62.0
Edge from 30 to 34, weight: 119.0
Edge from 30 to 35, weight: 96.0
Edge from 30 to 36, weight: 179.0
Edge from 30 to 37, weight: 192.0
Edge from 30 to 38, weight: 142.0
Edge from 30 to 39, weight: 79.0
Edge from 30 to 40, weight: 72.0
Edge from 30 to 41, weight: 115.0
Edge from 30 to 42, weight: 74.0


Edge from 31 to 31, weight: 0.0
Edge from 31 to 32, weight: 115.0
Edge from 31 to 33, weight: 46.0
Edge from 31 to 34, weight: 98.0
Edge from 31 to 35, weight: 75.0
Edge from 31 to 36, weight: 163.0
Edge from 31 to 37, weight: 179.0
Edge from 31 to 38, weight: 136.0
Edge from 31 to 39, weight: 67.0
Edge from 31 to 40, weight: 81.0
Edge from 31 to 41, weight: 129.0
Edge from 31 to 42, weight: 95.0
Edge from 32 to 32, weight: 0.0
Edge from 32 to 33, weight: 69.0
Edge from 32 to 34, weight: 86.0
Edge from 32 to 35, weight: 81.0
Edge from 32 to 36, weight: 60.0
Edge from 32 to 37, weight: 65.0
Edge from 32 to 38, weight: 54.0
Edge from 32 to 39, weight: 158.0
Edge from 32 to 40, weight: 195.0
Edge from 32 to 41, weight: 243.0
Edge from 32 to 42, weight: 190.0
Edge from 33 to 33, weight: 0.0
Edge from 33 to 34, weight: 71.0
Edge from 33 to 35, weight: 49.0
Edge from 33 to 36, weight: 117.0
Edge from 33 to 37, weight: 133.0
Edge from 33 to 38, weight: 98.0
Edge from 33 to 39, weight: 95.0
E

Dans le cas où le format n'est pas de type `FULL_MATRIX`, on renvoie un message d'erreur. 

In [ ]:
build_graph("/instances/stsp/dantzig42.tsp", "Graph_Test")

┌ Warning: edge weight format must be `FULL_MATRIX`
└ @ Main c:\Users\victo\OneDrive\Documents\Cours\MTH6412B\TP\Projet\Phase 1\Phase 1.ipynb:10


("Current format is :", "LOWER_DIAG_ROW")

### Question 7)

c.f fichier `main.jl` à l'url : https://github.com/nathanemac/Projet/tree/phase1/Phase%201